# Introducción a la Probabilidad y Estadística
En este cuaderno, exploraremos algunos de los conceptos que hemos discutido previamente. Muchos conceptos de probabilidad y estadística están bien representados en las principales bibliotecas para el procesamiento de datos en Python, como `numpy` y `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variables aleatorias y distribuciones
Comencemos extrayendo una muestra de 30 valores de una distribución uniforme de 0 a 9. También calcularemos la media y la varianza.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Para estimar visualmente cuántos valores diferentes hay en la muestra, podemos trazar el **histograma**:


In [ ]:
plt.hist(sample)
plt.show()

## Análisis de Datos Reales

La media y la varianza son muy importantes al analizar datos del mundo real. Carguemos los datos sobre jugadores de béisbol de [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Estamos utilizando un paquete llamado [**Pandas**](https://pandas.pydata.org/) aquí para análisis de datos. Hablaremos más sobre Pandas y cómo trabajar con datos en Python más adelante en este curso.

Vamos a calcular los valores promedio para edad, altura y peso:


In [ ]:
df[['Age','Height','Weight']].mean()

Ahora centrémonos en la altura y calculemos la desviación estándar y la varianza:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Además de la media, tiene sentido observar el valor mediano y los cuartiles. Se pueden visualizar usando un **diagrama de caja**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

También podemos hacer diagramas de caja de subconjuntos de nuestro conjunto de datos, por ejemplo, agrupados por rol del jugador.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Nota**: Este diagrama sugiere que, en promedio, las alturas de los primeros bases son mayores que las alturas de los segundos bases. Más adelante aprenderemos cómo podemos probar esta hipótesis de manera más formal y cómo demostrar que nuestros datos son estadísticamente significativos para mostrarlo.  

La edad, la altura y el peso son todas variables aleatorias continuas. ¿Qué crees que es su distribución? Una buena manera de averiguarlo es trazar el histograma de valores: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribución Normal

Vamos a crear una muestra artificial de pesos que siga una distribución normal con la misma media y varianza que nuestros datos reales:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Dado que la mayoría de los valores en la vida real se distribuyen normalmente, no deberíamos usar un generador de números aleatorios uniforme para generar datos de muestra. Esto es lo que sucede si intentamos generar pesos con una distribución uniforme (generada por `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervalos de confianza

Ahora calculemos intervalos de confianza para los pesos y las alturas de los jugadores de béisbol. Usaremos el código [de esta discusión en stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Pruebas de hipótesis

Exploremos diferentes roles en nuestro conjunto de datos de jugadores de béisbol:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Vamos a probar la hipótesis de que los primeros base son más altos que los segundos base. La forma más sencilla de hacer esto es probar los intervalos de confianza:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Podemos ver que los intervalos no se superponen.

Una forma estadísticamente más correcta de probar la hipótesis es usar un **test t de Student**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Los dos valores devueltos por la función `ttest_ind` son:
* El valor p puede considerarse como la probabilidad de que dos distribuciones tengan la misma media. En nuestro caso, es muy bajo, lo que significa que hay una fuerte evidencia que respalda que los primera base son más altos.
* El valor t es el valor intermedio de la diferencia de medias normalizada que se utiliza en la prueba t, y se compara con un valor umbral para un valor de confianza dado.


## Simulando una Distribución Normal con el Teorema del Límite Central

El generador pseudoaleatorio en Python está diseñado para darnos una distribución uniforme. Si queremos crear un generador para distribución normal, podemos usar el teorema del límite central. Para obtener un valor distribuido normalmente simplemente calcularemos la media de una muestra generada uniformemente.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Correlación y Evil Baseball Corp

La correlación nos permite encontrar relaciones entre secuencias de datos. En nuestro ejemplo ficticio, imaginemos que hay una corporación malvada de béisbol que paga a sus jugadores según su altura: cuanto más alto es el jugador, más dinero recibe. Supongamos que hay un salario base de $1000 y un bono adicional de $0 a $100, dependiendo de la altura. Tomaremos a los jugadores reales de la MLB y calcularemos sus salarios imaginarios:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Ahora calculemos la covarianza y la correlación de esas secuencias. `np.cov` nos dará una llamada **matriz de covarianza**, que es una extensión de la covarianza a múltiples variables. El elemento $M_{ij}$ de la matriz de covarianza $M$ es una correlación entre las variables de entrada $X_i$ y $X_j$, y los valores diagonales $M_{ii}$ son la varianza de $X_{i}$. De manera similar, `np.corrcoef` nos dará la **matriz de correlación**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Una correlación igual a 1 significa que hay una fuerte **relación lineal** entre dos variables. Podemos ver visualmente la relación lineal trazando un valor contra el otro:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Veamos qué pasa si la relación no es lineal. Supongamos que nuestra corporación decidió ocultar la obvia dependencia lineal entre alturas y salarios, e introdujo cierta no linealidad en la fórmula, como `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

En este caso, la correlación es un poco menor, pero todavía es bastante alta. Ahora, para hacer que la relación sea aún menos obvia, podríamos querer agregar un poco de aleatoriedad extra añadiendo alguna variable aleatoria al salario. Veamos qué sucede:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> ¿Puedes adivinar por qué los puntos se alinean en líneas verticales así?

Hemos observado la correlación entre un concepto artificialmente creado como el salario y la variable observada *altura*. Veamos también si las dos variables observadas, como la altura y el peso, también se correlacionan:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Desafortunadamente, no obtuvimos ningún resultado, solo algunos valores extraños `nan`. Esto se debe a que algunos de los valores en nuestra serie están indefinidos, representados como `nan`, lo que provoca que el resultado de la operación también sea indefinido. Al observar la matriz podemos ver que `Weight` es la columna problemática, porque se ha calculado la autocorrelación entre los valores de `Height`.

> Este ejemplo muestra la importancia de la **preparación de datos** y la **limpieza**. Sin datos adecuados no podemos calcular nada.

Usemos el método `fillna` para completar los valores faltantes y calcular la correlación: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

De hecho, existe una correlación, pero no tan fuerte como en nuestro ejemplo artificial. En efecto, si observamos el diagrama de dispersión de un valor frente al otro, la relación sería mucho menos obvia:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusión

En este cuaderno hemos aprendido cómo realizar operaciones básicas sobre datos para calcular funciones estadísticas. Ahora sabemos cómo usar un aparato sólido de matemáticas y estadísticas para probar algunas hipótesis, y cómo calcular intervalos de confianza para variables arbitrarias dado un conjunto de datos.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por la precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de ningún malentendido o interpretación errónea que pueda surgir del uso de esta traducción.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
